_Note:_ _family_ and _set_ both refer to unordered sets, though the former is used as a set-of-sets.

**Definition:** $[n]=\{0, 1, \ldots, n-1\}$.

**Definition:** A family of sets `S` is called _union-closed_ if for every $T\subseteq S$, we have

\\[
\bigcup_{e\in T} e \in S
\\]

**Definition:** For a family of sets, the _element-family_ $S_x$ is defined as:
\\[
S_x =  \big \{ s\in S: x \in s \big \}
\\]
The _element count_ is defined as $|S_x|$.

**Conjecture (Frankl):** Every union-closed family of sets has an element in at least half the sets. This bound is sharp, since it is realized by powersets of $[n]$.

In [31]:
from typing import Set, Dict
from functools import reduce
from operator import or_
import builtins
from collections import Counter


def get_universe(sets: Set[frozenset]) -> frozenset:
    return reduce(or_, sets, frozenset())


def union_close(sets: Set[frozenset]) -> Set[frozenset]:
    closed = set(sets) | {frozenset()}  # Add the empty set, which corresponds to an empty union.
    iteration_needed = True
    while iteration_needed:
        to_add = set()
        for member in sets:
            for other_set in closed:
                union = member | other_set
                if union not in closed:
                    to_add.add(union)
        iteration_needed = bool(to_add)
        closed.update(to_add)
    return closed

def element_counts(sets: Set[frozenset]) -> Counter:
    return Counter(element for member in sets for element in member)

def element_proportions(sets: Set[frozenset]) -> Dict[int, float]:
    number = len(sets)
    return {element: count / number for element, count  in element_counts(sets).items()}

In [25]:
assert len(union_close({frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4})})) == 2 **4

## Example

Build an example with some interesting combinatorics. We build a separable family of sets generated by triples from 4-element subsets of the set $[16]$. Note that the union of any two sets from the 4-element subsets equals the 4-element set, so we can only form sets of size which can be expressed as a sum of 3's and 4's, i.e.:

- There are $12$ $3$-element sets.
- There are $4$ $4$-element sets.
- There are no $5$-element sets.
- There are ${4 \choose 2}\cdot 3^2=6 \cdot 3^2=54$ $6$-element sets (selecting one from each 4-element family).
- There are $4 \cdot 3 \cdot 3=36$ $7$-element sets.
- There are ${4 \choose 2}=6$ $8$-element sets.
- There are ${4 \choose 3}\cdot 3^3=108$ $9$-element sets.
- There are $4\cdot {3 \choose 2} \cdot 3^2=108$ $10$-element sets.
- There are ${4 \choose 2} \cdot {2 \choose 1} \cdot 3=6\cdot 2 \cdot 3=36$ $11$-element sets.
- There are $3^4 + {4 \choose 3} = 85$ $12$-element sets.
- There are ${4 \choose 3}\cdot 3^3=108$ $12$-element sets.
- There are ${4 \choose 1}\cdot 3^3=4\cdot 27=108$ $13$-element sets.
- There are ${4 \choose 2}\cdot 3^2=54$ $14$-element sets.
- There are ${4 \choose 1}\cdot 3=12$ $15$-element sets.
- ... and so on.

Unfortunately, since there are more ways to express larger sets as sums of 3's and 4's, there are too many large sets.



In [85]:
base_sets = set()
for start in range(0, 16, 4):
    a, b, c, d = range(start, start + 4)
    base_sets.update([
        frozenset({a, b, c}),
        frozenset({a, b, d}),
        frozenset({a, c, d}),
    ])
example = union_close(base_sets)
counts = element_counts(example)
len(base_sets)
def assert_lengths(example, set_size, expected):
    assert len([s for s in example if len(s) == set_size]) == expected

assert_lengths(example, 1, 0)
assert_lengths(example, 2, 0)
assert_lengths(example, 3, 12)
assert_lengths(example, 3, 12)
assert_lengths(example, 4, 4)
assert_lengths(example, 5, 0)
assert_lengths(example, 6, 54)
assert_lengths(example, 7, 36)
assert_lengths(example, 8, 6)
assert_lengths(example, 9, 108)
assert_lengths(example, 10, 108)
assert_lengths(example, 11, 36)
assert_lengths(example, 12, 85)
assert_lengths(example, 13, 108)
assert_lengths(example, 14, 54)
assert_lengths(example, 15, 12)

EXAMPLES = {'3-element sets from 4 element families from [16]': example}
element_proportions(example)

{4: 0.8,
 5: 0.6,
 6: 0.6,
 7: 0.6,
 8: 0.8,
 9: 0.6,
 10: 0.6,
 11: 0.6,
 12: 0.8,
 14: 0.6,
 15: 0.6,
 0: 0.8,
 1: 0.6,
 2: 0.6,
 3: 0.6,
 13: 0.6}

Can we generalize this example? Let's try building $k-1$ sets of length $k-1$ from $n/k$ subsets of the universe:

In [88]:
def get_example_base_sets(k: int, n: int) -> Set[frozenset]:
    assert n % k == 0
    base_sets = set()
    for start in range(0, n, k):
        subset = frozenset(range(start, start + k))
        for excluded in range(start + 1, start + k):
            base_sets.add(subset - {excluded})
    return base_sets

def build_example(k: int, n: int) -> Set[frozenset]:
    return union_close(get_example_base_sets(k, n))


assert build_example(4, 16) == EXAMPLES['3-element sets from 4 element families from [16]']

In [89]:
def print_example(k, n):
    print(f'k={k}, n={n}, min proportion={min(element_proportions(build_example(k, n)).values())}')

print_example(4, 16)    
print_example(4, 20)
print_example(4, 24)
print_example(6, 12)
print_example(6, 18)
print_example(6, 24)
print_example(9, 18)
print_example(7, 21)
print_example(5, 20)
print_example(5, 10)
print_example(3, 21)

k=4, n=16, min proportion=0.6
k=4, n=20, min proportion=0.6
k=4, n=24, min proportion=0.6
k=6, n=12, min proportion=0.7142857142857143
k=6, n=18, min proportion=0.7142857142857143
k=6, n=24, min proportion=0.7142857142857143
k=9, n=18, min proportion=0.8
k=7, n=21, min proportion=0.75
k=5, n=20, min proportion=0.6666666666666666
k=5, n=10, min proportion=0.6666666666666666
k=3, n=21, min proportion=0.5


So it appears that the minimum proportion depends only on $k$, and that the minimum proportion is achieved for $k=3$ at precisely the Frankl bound of $\frac{1}{2}$. *TODO*: prove this?

What happens if we combine these examples into a different partition?

In [118]:
from typing import List, Tuple
def get_combined_example_base_sets(k_ns: List[Tuple[int, int]]) -> Set[frozenset]:
    acc = 0
    base_sets = set()
    for k, n in k_ns:
        current_base_sets = get_example_base_sets(k, n)
        base_sets.update(frozenset(i + acc for i in base_set) for base_set in current_base_sets)
        acc += n
    return base_sets

def build_combined_example(k_ns: List[Tuple[int, int]]) -> Set[frozenset]:
    return union_close(get_combined_example_base_sets(k_ns))

def print_combined_example(k_ns):
    partition_string = f'{sum(k * n for k, n in k_ns)}={"+".join(f"{k}*{n // k}" for k, n in k_ns)}'
    example = build_combined_example(k_ns)
    print(f'{partition_string}, min proportion={min(element_proportions(example).values())}, max proportion={max(element_proportions(example).values())}')

print_combined_example([(4, 8), (5, 10)])
print_combined_example([(4, 12), (5, 10)])
print_combined_example([(3, 6), (4, 8), (5, 10)])
print_combined_example([(4, 8), (5, 10), (6, 12)])

82=4*2+5*2, min proportion=0.6, max proportion=0.8333333333333334
98=4*3+5*2, min proportion=0.6, max proportion=0.8333333333333334
100=3*2+4*2+5*2, min proportion=0.5, max proportion=0.8333333333333334
154=4*2+5*2+6*2, min proportion=0.6, max proportion=0.8571428571428571


So it appears that in this family of examples, the minimum and max proportion is controlled by the smallest and largest $k$ respectively.